## Problem

In OpenQAOA there are several problems that are implemented in order to perform the Ising model, in this example we will address a specific case of  the [Knapsack problem](https://en.wikipedia.org/wiki/Knapsack_problem). The knapsack problem asks us to find a combination of items such that the total weight is within the capacity of the knapsack and maximize the total value of the items.

### Classical Method

The knapsack problem has a set of items  $N^{\prime}=\{1, \ldots, N\}$, consisting of $N$ items, the $j$-th with profit $p_{j}$ and weight $w_{j}$, and the capacity value $W$. Then, the objective is to select a subset of $N^{\prime}$ such that the total profit of the selected items is maximized and the total weight does not exceed $W$.
$$
\begin{aligned}
\operatorname{maximizar} & \sum_{j=1}^{N} p_{j} x_{j} \\
\text { sujeto a } & \sum_{j=1}^{N} w_{j} x_{j} \leq W \\
& x_{j} \in\{0,1\}, \quad j=1, \ldots, N
\end{aligned}
$$
Let us denote the optimal solution vector by $x^{*}=\left(x_{i}, \ldots, x_{N}\right)$ .

<div class="alert alert-block alert-warning"> 

Example consider ** 4 items, where the weights are {3, 4, 6, 5}, the values are {2, 3, 1, 4}, and the  weight_capacity is 8, **, try to find the best combination

</div>

This is possible to solve using dynamic programming and one good tool is using a matrix, you can see for this example the oslution in the above Table, where columns represent the weight(8),and the rows represent the profits and weights of items. 


<table  style="width:100%;color: white;">
    <thead >
        <tr style=" background-color: #2f3974;">
            <th>Item </th>
            <th>0</th>
            <th>1</th>
            <th>2</th>
            <th>3</th>
            <th>4</th>
            <th>5</th>
            <th>6</th>
            <th>7</th>
            <th>8</th>
        </tr>
    </thead>
    <tbody>
        <tr style=" background-color: #3a6486;">
            <td>0</td>
            <td>0</td>
            <td>0</td>
            <td>0</td>
            <td>0</td>
            <td>0</td>
            <td>0</td>
            <td>0</td>
            <td>0</td>
            <td>0</td>
        </tr>
        <tr style=" background-color: #3a6486;">
            <td>1</td>
            <td>0</td>
            <td>0</td>
            <td>0</td>
            <td>2</td>
            <td>2</td>
            <td>2</td>
            <td>2</td>
            <td>2</td>
            <td>2</td>
        </tr>
        <tr style=" background-color: #3a6486;">
            <td>2</td>
            <td>0</td>
            <td>0</td>
            <td>0</td>
            <td>2</td>
            <td>3</td>
            <td>3</td>
            <td>3</td>
            <td>5</td>
            <td>5</td>
        </tr>
        <tr style=" background-color: #3a6486;">
            <td>3</td>
            <td>0</td>
            <td>0</td>
            <td>0</td>
            <td>2</td>
            <td>3</td>
            <td>3</td>
            <td>3</td>
            <td>5</td>
            <td>5</td>
        </tr>
        <tr style=" background-color: #3a6486;">
            <td>4</td>
            <td>0</td>
            <td>0</td>
            <td>0</td>
            <td>2</td>
            <td>3</td>
            <td>3</td>
            <td>4</td>
            <td>5</td>
            <td style=" background-color: #4fbeae;">5</td>
        </tr>
    </tbody>
 </table>

<center>Table 1. Solution for the knapsack Problem's example</center>


Table 1 shows that  of the $ 2^4 =  16$ combination the best combination is the state `1100`

### Design  the problem

The necessary dependencies to build the problem have to be initialized, in order to display the graphs of the results and the necessary OpenQAOA objects and methods to generate the quantum algorithm.

In [1]:
# Methods to use in the  notebook
%load_ext autoreload
%autoreload 2
from IPython.display import clear_output

# Libaries to manipulate, print, plot, and save the data.
import matplotlib.pyplot as plt
import numpy as np

# Random generator for the problem
from random import seed,randrange, randint

# OpenQAOA libraries to design, execute the algorithm in different backends and optimisers
from openqaoa.problems.problem import Knapsack
from openqaoa.workflows.optimizer import QAOA 
from openqaoa.devices import create_device
from openqaoa.optimizers.qaoa_optimizer import available_optimizers
from openqaoa.utilities import ground_state_hamiltonian
# Import the method to plot the graph 
from openqaoa.utilities import plot_graph

Using the method random we can generate a random list of four elements, from which the Partition problem is to be implemented.

In [2]:
np.random.seed(1)
num_items = 3 # number of items
weight_capacity = 10 # max weight of a bin
penalty = 1.0

values = np.random.randint(1, weight_capacity, num_items)
weights = np.random.randint(1, weight_capacity, num_items)

print(values,weights,weight_capacity,penalty)

[6 9 6] [1 1 2] 10 1.0


####  Number Partition problem to QAOA

In order to work the problem you want to design you must convert it to Quadratic Unconstrained Binary Optimization (QUBO) format. That is possible using the Class *NumberPartitio*, where this has one parameter, an integer list, one object of this class has the method `get_pubo_problem()`, that translates the problem into a QUBO problem.

In [3]:
qubo = Knapsack(values, weights, weight_capacity,penalty).get_qubo_problem()
qubo.asdict()

{'terms': [[0, 1],
  [0, 2],
  [0, 3],
  [1, 2],
  [1, 3],
  [2, 3],
  [4, 5],
  [4, 6],
  [5, 6],
  [0, 4],
  [0, 5],
  [0, 6],
  [1, 4],
  [1, 5],
  [1, 6],
  [2, 4],
  [2, 5],
  [2, 6],
  [3, 4],
  [3, 5],
  [3, 6],
  [0],
  [1],
  [2],
  [3],
  [4],
  [5],
  [6]],
 'weights': [1.0,
  2.0,
  4.0,
  4.0,
  8.0,
  16.0,
  0.5,
  1.0,
  1.0,
  0.5,
  0.5,
  1.0,
  1.0,
  1.0,
  2.0,
  2.0,
  2.0,
  4.0,
  4.0,
  4.0,
  8.0,
  0.5,
  1.0,
  2.0,
  4.0,
  3.5,
  5.0,
  4.0],
 'constant': 12.5,
 'n': 7}

Implement the QAOA quantum algorithm  from the OpenQAOA implementation

In [4]:
# In OpenQAOA you can use different devices either local 
# or from different cloud backends such as Qiskit, Pyquil.
sim = create_device(location='local', name='vectorized')

# Init the object QAOA and certain of its properties can be modified
q = QAOA()
q.set_device(sim)
q.compile(qubo) 

# Execute the quantum algorithm
q.optimize()

#### Results

You can print different things from the QAOA object in the section `q.results` as:

and check the `solutions_bitstrings` and the exactly answer with `ground_state_hamiltonian`

In [5]:
print("solution bitstring: ",q.results.most_probable_states['solutions_bitstrings'])
print("ground state: ",ground_state_hamiltonian(q.cost_hamil)[1])

solution bitstring:  ['0010111']
ground state:  ['0110111']


### Try your

Consider the procces from the previous example we analize different results using all the optimizers that OpenQAOA support, this divide in *'scipy'* and *'custom_scipy_gradient'*, as:

In [6]:
optimisers = available_optimizers()['scipy']+ available_optimizers()['custom_scipy_gradient']
optimisers

['nelder-mead',
 'powell',
 'cg',
 'bfgs',
 'newton-cg',
 'l-bfgs-b',
 'tnc',
 'cobyla',
 'slsqp',
 'trust-constr',
 'dogleg',
 'trust-ncg',
 'trust-exact',
 'trust-krylov',
 'vgd',
 'newton',
 'rmsprop',
 'natural_grad_descent',
 'spsa']

Using 19 optimisers, for all those you can choose the method to compute the Jacobian and Hessian, for Jacobian you can consider all the  methods and Hessian only *finite_difference*.

Define the Jacobian list:

In [7]:
jac = ['finite_difference', 'param_shift','grad_spsa', 'stoch_param_shift']

Define the Hessian value:

In [8]:
hess = jac[0]